# Day 3

--- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

## Part 2

--- Part Two ---
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [ ]:
test_input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

In [ ]:
from dataclasses import dataclass, field
from typing import List, Tuple, Set, Iterable
import re
from tabulate import tabulate

# "." | "#" | "123" | ...
PartNumber = str
Symbol = str
PartNumberOrSymbol = PartNumber | Symbol
# "." "123" "123" "123" "." "."
Row = List[PartNumberOrSymbol]

AmountRows = int
AmountCols = int

GridCoordinates = Tuple[int, int] # row, col

@dataclass
class ConnectedDigit():
    row: int
    col: int
    part_no: PartNumber
    connected_by: List[Symbol]

ConnectedDigitRow = List[ConnectedDigit]

@dataclass 
class ConnectedGear():
    row: int
    col: int
    connected_to: List[str] = field(init=False, default_factory=list) # Gear part_no:s
    def connect(self, part_no):
        self.connected_to.append(part_no)

ConnectedGearRow = List[ConnectedDigit]

@dataclass
class Schematic():
    # Part numbers can occupy multiple cells, so values are duplicated per cell in this repr
    lookup_grid_size: Tuple[AmountRows, AmountCols]
    lookup_grid: List[Row] = field(init=False) # Post-inited with lengths
    symbol_coordinates: Set[GridCoordinates] = field(init=False, default_factory=set)
    connected_digits: List[ConnectedDigitRow] = field(init=False, default_factory=list) # Post-inited with lengths
    connected_gears: List[ConnectedGearRow] = field(init=False, default_factory=list) # Post-inited with lengths

    def replace_symbol(self, s: str, col: int, row: int):
        """Replace a string in the lookup grid"""
        self.lookup_grid[row][col] = s
        #if s in SYMBOLS:
        if not s.isdigit():
            #print(f"added symbol {(row, col)}")
            self.symbol_coordinates.add((row, col))


    def find_neighbor_part_numbers(self, row, col, verbose = False) -> List[ConnectedDigit]:
        """Finds all moore neighbors that are numbers.
        Pass verbose=True for visualization of the symbol lookups.
        Returns findings."""
        found: List[ConnectedDigit] = []
        pattern = re.compile(r"\d+")
        viz = []
        for offset_row in range(-1, 2):
            target_row = row + offset_row
            if not (target_row >= 0 and target_row < len(self.lookup_grid)):
                # Outside rows
                viz.append(["?", "?", "?"])
                continue

            row_viz = []
                
            for offset_col in range(-1, 2):
                target_col = col + offset_col
                if not (target_col >= 0 and target_col < len(self.lookup_grid[target_row])):
                    # Outside columns
                    row_viz.append("?")
                    continue
                    
                if target_row == row and target_col == col:
                    # Skip searched, only neighbors
                    row_viz.append(self.lookup_grid[target_row][target_col]) # symbol
                    continue
                    
                value = self.lookup_grid[target_row][target_col]

                row_viz.append(value)
                
                if pattern.match(value) is None:
                    # Not a number
                    continue

                #print("found", target_row, target_col, value
                cd = ConnectedDigit(target_row, target_col, value, self.lookup_grid[target_row][target_col])
                found.append(cd)

            viz.append(row_viz)

        #print(f"Around {row} {col}:")
        if verbose:
            print(tabulate(viz))
        
        return found

    def update_connections(self, verbose = False) -> List[PartNumberOrSymbol]:
        """Update the internal connection cache. Look up results from self.connected_digits.
        Only considers symbols for connection condition.
        Pass verbose=True for visualization of the symbol lookups
        Returns the resulting self.connected_digits."""
        #print(self.symbol_coordinates)
        #debug_symbols = set()
        for coords in self.symbol_coordinates:
            symbol_row, symbol_col = coords
            #debug_symbols.add(self.lookup_grid[symbol_row][symbol_col])
            found = self.find_neighbor_part_numbers(symbol_row, symbol_col, verbose)
            
            if verbose:
                print("Found:", found, "\n")
                
            #print(f"all found for {symbol_row, symbol_col}: {found}")
            for cd in found:
                sym = self.lookup_grid[symbol_row][symbol_col]
                if sym == "*": # gear
                    if self.connected_gears[symbol_row][symbol_col] is None:
                        self.connected_gears[symbol_row][symbol_col] = ConnectedGear(symbol_row, symbol_col)
                    
                    self.connected_gears[symbol_row][symbol_col].connect(cd.part_no)

                #if self.connected_digits[cd.row][cd.col] is not None:
                #    continue
                
                self.connected_digits[cd.row][cd.col] = cd

        #print(self.connected_digits)
        #print(self.connected_gears)
        #print(debug_symbols)
        return self.connected_digits

    
    def __post_init__(self):
        """Initialize the grid with size given in constructor"""
        print(f"creating grid of size (rows /cols): {self.lookup_grid_size[0]} {self.lookup_grid_size[1]}")
        self.lookup_grid =      [["." for _ in range(self.lookup_grid_size[1])] for _ in range(self.lookup_grid_size[0])]
        self.connected_digits = [[None for _ in range(self.lookup_grid_size[1])] for _ in range(self.lookup_grid_size[0])]
        self.connected_gears = [[None for _ in range(self.lookup_grid_size[1])] for _ in range(self.lookup_grid_size[0])]


def parse_input(input: str) -> Schematic:
    """Parse puzzle input into a processable data type"""
    rows = input.splitlines()
    longest = len(max(rows, key=len))
    schematic = Schematic(lookup_grid_size=(len(rows), longest))
    pattern = re.compile(rf"\.*(\d+|\D)\.*")
    #print(pattern)
    for i, row in enumerate(rows):
        matches = pattern.finditer(row)
        #print(row)
        #print([m for m in matches])
        for m in matches:
            s = m.group(1)
            start, end = m.span(1)
            for col in range(start, end):
                #print(s, col, i)
                schematic.replace_symbol(s, col, i)

    return schematic

In [ ]:
import itertools

def solve(schematic: Schematic) -> int:
    """Solve a schematic for part 1"""
    #print(schematic)
    connections = schematic.update_connections(verbose=False)
    deduped = []
    gear_ratios: [Tuple(int, int)] = []
    
    prev = ""
    for row in connections:
        for col in row:
            part_no = col.part_no if col is not None else ""
            if part_no != "" and prev != part_no:
                deduped.append(col)
            prev = part_no
    #print(deduped)

    gear_deduped = []
    for row in schematic.connected_gears:
        for col in row:
            if col is None:
                continue

            if len(set(col.connected_to)) != 2:
                continue
            
            gear_deduped.append(list(set(col.connected_to)))
            
    #print(gear_deduped)

    return sum([int(c.part_no) for c in deduped]), sum([int(gear_combo[0]) * int(gear_combo[1]) for gear_combo in gear_deduped])

In [ ]:
test_input_schematic = parse_input(test_input)
print("Solution for test input:", solve(test_input_schematic))

In [ ]:
input_lines: []
with open("../input.txt", "r") as f:
    input_lines = f.read()
test_input_schematic = parse_input(input_lines)
print("Solution for part 1 and 2:", solve(test_input_schematic))